# 👚 02_segmentation.ipynb — Pipeline de segmentation robuste et traçable

Ce notebook traite la **segmentation vestimentaire** sur un lot d’images, en contrôlant à chaque étape la robustesse du pipeline (sécurité du token, accès aux fichiers, appel API, génération des masques et sauvegarde du résultat).  
Chaque étape déclare un booléen OK/NOK, et un rapport final en synthétise l'état.  
Les chemins sont adaptés à ta structure :  
- Images sources : `../data/raw/images/`
- Masques annotés : `../data/raw/annotations/`
- Images/masques générés : `../data/processed/`

In [26]:
# 1. Chargement du token Hugging Face centralisé via utils.py

import sys
from pathlib import Path

# Ajoute le dossier src/ au PYTHONPATH si ce n'est pas déjà fait
sys.path.append(str((Path.cwd().parent / "src").resolve()))

from fashion_trend_intelligence.utils import HF_TOKEN

ok_token = HF_TOKEN is not None and HF_TOKEN.startswith("hf_")
if ok_token:
    print("✅ Token Hugging Face chargé via utils.py :", HF_TOKEN[:10], "…")
else:
    print("❌ Token HF absent ou mal formaté (ou .env introuvable).")

✅ Token Hugging Face chargé via utils.py : hf_pLHNzBu …


In [27]:
# 2. Vérification des dépendances maîtresses
ok_dependencies = True
for pkg in ['requests', 'dotenv', 'numpy', 'matplotlib', 'PIL', 'huggingface_hub']:
    try:
        __import__(pkg)
    except ImportError:
        ok_dependencies = False
        print(f"❌ Package manquant : {pkg}")
if ok_dependencies:
    print("✅ Toutes les dépendances principales sont installées.")

✅ Toutes les dépendances principales sont installées.


In [28]:
# 3. Chargement d’une image brute et de son masque d’annotation
from PIL import Image

img_dir = Path("../data/raw/images")
mask_dir = Path("../data/raw/annotations")
img_files = sorted(img_dir.glob("*.png"))
mask_files = sorted(mask_dir.glob("*.png"))

ok_img = ok_mask = False
try:
    img_path = img_files[0]  # Prendre la première image trouvée pour le test
    mask_path = mask_files[0]  # Prendre le premier masque pour le test
    img = Image.open(img_path)
    mask = Image.open(mask_path)
    ok_img = ok_mask = True
    print(f"✅ Image test chargée : {img_path}")
    print(f"✅ Masque annoté chargé : {mask_path}")
except Exception as e:
    print(f"❌ Erreur chargement image ou masque : {e}")
    ok_img, ok_mask = False, False

✅ Image test chargée : ../data/raw/images/image_0.png
✅ Masque annoté chargé : ../data/raw/annotations/mask_0.png


In [29]:
# 4. Test API SegFormer sur l’image
ok_api_call = False
if ok_img and ok_token and ok_dependencies:
    import requests
    model_id = "sayeed99/segformer_b3_clothes"
    with open(img_path, "rb") as f:
        image_bytes = f.read()
    headers = {"Authorization": f"Bearer {HF_TOKEN}", "Content-Type": "image/png"}
    try:
        resp = requests.post(
            f"https://api-inference.huggingface.co/models/{model_id}",
            headers=headers,
            data=image_bytes,
            timeout=20,
        )
        if resp.status_code in (200, 503):
            ok_api_call = True
            print(f"✅ API appelée : code {resp.status_code}")
        else:
            print(f"❌ Code retour API inattendu : {resp.status_code} ({resp.text[:120]})")
    except Exception as e:
        print(f"❌ Erreur appel API : {e}")
else:
    print("⏭️  Appel API sauté (pré-requis non validés).")

✅ API appelée : code 200


In [30]:
# 5. Décodage du résultat, création et sauvegarde du masque
ok_processing = False
out_dir = Path("../data/processed")
out_dir.mkdir(parents=True, exist_ok=True)
result_mask_path = out_dir / f"{img_path.stem}_mask_pred.png"
try:
    if ok_api_call:
        import base64
        from io import BytesIO
        output = resp.json()
        first_mask = next((elt for elt in output if "mask" in elt), None)
        if first_mask is not None:
            mask_b64 = first_mask["mask"]
            mask_bytes = base64.b64decode(mask_b64)
            mask_img = Image.open(BytesIO(mask_bytes))
            mask_img.save(result_mask_path)
            ok_processing = True
            print(f"✅ Masque prédictif sauvegardé dans {result_mask_path}")
        else:
            print("❌ Aucun masque dans la réponse API.")
    else:
        print("⏭️  Traitement du mask sauté (appel API non ok).")
except Exception as e:
    ok_processing = False
    print(f"❌ Erreur lors du décodage/sauvegarde du masque : {e}")

✅ Masque prédictif sauvegardé dans ../data/processed/image_0_mask_pred.png


In [31]:
# Bonus : visualisation rapide d’un masque de segmentation
import base64
from PIL import Image
from io import BytesIO

mask_b64 = output[0]["mask"]
mask_bytes = base64.b64decode(mask_b64)
mask_img = Image.open(BytesIO(mask_bytes))
mask_img.show()
print("Premier masque décodé et affiché (base64 → PNG).")

Premier masque décodé et affiché (base64 → PNG).


In [32]:
# 6. Rapport global de validation du pipeline
from IPython.display import display, Markdown

tests = [
    ("Token Hugging Face chargé", ok_token),
    ("Dépendances installées", ok_dependencies),
    ("Image brute chargée", ok_img),
    ("Masque annoté chargé", ok_mask),
    ("Appel API SegFormer (200/503)", ok_api_call),
    ("Masque prédictif sauvegardé", ok_processing),
]

rapport_md = "### Rapport validation segmentation<br/>\n\n"
for label, status in tests:
    icon = "✅" if status else "❌"
    color = "green" if status else "red"
    rapport_md += f"- <span style='color:{color}; font-weight:bold'>{icon} {label}</span><br/>\n"

all_ok = all(status for _, status in tests)
if all_ok:
    rapport_md += "\n---\n<span style='color:green; font-weight:bold; font-size:1.1em;'>✅ Pipeline complet fonctionnel : images, masques, API et sauvegarde !</span>"
else:
    rapport_md += "\n---\n<span style='color:red; font-weight:bold; font-size:1.1em;'>❌ Un ou plusieurs tests ont échoué. Voir ci-dessus.</span>"

display(Markdown(rapport_md))

### Rapport validation segmentation<br/>

- <span style='color:green; font-weight:bold'>✅ Token Hugging Face chargé</span><br/>
- <span style='color:green; font-weight:bold'>✅ Dépendances installées</span><br/>
- <span style='color:green; font-weight:bold'>✅ Image brute chargée</span><br/>
- <span style='color:green; font-weight:bold'>✅ Masque annoté chargé</span><br/>
- <span style='color:green; font-weight:bold'>✅ Appel API SegFormer (200/503)</span><br/>
- <span style='color:green; font-weight:bold'>✅ Masque prédictif sauvegardé</span><br/>

---
<span style='color:green; font-weight:bold; font-size:1.1em;'>✅ Pipeline complet fonctionnel : images, masques, API et sauvegarde !</span>